<a href="https://colab.research.google.com/github/Vandrade85/FIAP_data_engineering_bootcamp/blob/main/Bootcamp_Data_Engineering_Exercicios_Van.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Temos duas bases de dados pessoais (fictícios)


*   cadastro.csv (dados de cadastro)
*   dados_medicos.csv (registros médicos)

Temos que unificar as duas bases e criar uma nova base agrupada

Código localizado em https://github.com/renzoziegler/data_engineering_bootcamp


Exercício 1:


*   Importar o Pandas
*   Instalar e importar a biblioteca fuzzywuzzy (será importante mais pra frente)
*   Ler os arquivos CSV fornecidos
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/cadastro.csv
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/dados_medicos.csv
*   Realizar uma análise exploratória dos dados



In [50]:
## Instalação das bibliotecas
!pip install pandas
!pip install fuzzywuzzy

In [51]:
## Importação das bibliotecas
import pandas as pd
from fuzzywuzzy import fuzz

Para análise exploratória, vamos entender as bases de dados: seu tamanho, quais dados estão disponíveis, qual o formato deles, se estão completos ou não.....

In [94]:
# URLs dos arquivos no GitHub
url_cadastro = "https://raw.githubusercontent.com/Vandrade85/FIAP_data_engineering_bootcamp/main/cadastro.csv"
url_dados_medicos = "https://raw.githubusercontent.com/Vandrade85/FIAP_data_engineering_bootcamp/main/dados_medicos.csv"

# Carregar os arquivos em dataframe
df_cadastro = pd.read_csv(url_cadastro)
df_dados_medicos = pd.read_csv(url_dados_medicos)

# Exibir as primeiras 5 linhas
print("Dados de Cadastro (primeiras 5 linhas):")
print(df_cadastro.head().to_string(index=False))

print("\nDados Médicos (primeiras 5 linhas):")
print(df_dados_medicos.head().to_string(index=False))


Dados de Cadastro (primeiras 5 linhas):
                           nome  data_nasc         cpf                    mae   celular                                                                 email  telefone_fixo
            Cauã Isaac Caldeira 02-03-1945 35083528118 Vitória Kamilly Sueli  989048340               cauaisaaccaldeira__cauaisaaccaldeira@hardquality.com.br       28317237
    Lúcia Carolina Luzia Almada 11-11-1946  6480417194     Stefany Teresinha  997275887                                luciacarolinaluziaalmada_@santarte.com       27446850
    Vicente Marcos Edson Castro 15-01-1959 72192158432 Cecília Laura Larissa  994560743                          vicentemarcosedsoncastro-74@lavorosjc.com.br       28581372
    Yago Pedro Henrique da Cruz 24-10-1945 68328310562      Cristiane Pietra  982340551                      yagopedrohenriquedacruz_@elevaconsultancy.com.br       25189906
Valentina Stefany Alana Cardoso 23-03-1953 76298423761     Francisca Adriana  999976430 valenti

In [68]:
# Qdt linhas, qdt colunas
print(df_cadastro.shape)
print(df_dados_medicos.shape)

(393, 7)
(368, 6)


In [69]:
#Possiveis combinaçoes
resultado = 393 * 368 ## Qdt de linhas do cadastro x a qdt de linhas dos dados medicos
print(resultado)

144624


In [70]:
print("\nCadastro")
print(df_cadastro.dtypes)

print("\nDados Medicos")
print(df_dados_medicos.dtypes)


Cadastro
nome             object
data_nasc        object
cpf               int64
mae              object
celular           int64
email            object
telefone_fixo     int64
dtype: object

Dados Medicos
nome              object
data_nasc         object
cpf                int64
tipo_sanguineo    object
peso               int64
altura            object
dtype: object


Exercício 2: Pré-processamento
*   Quais tarefas vocês listam como necessárias?
 *  Padronização de campos
 *  Substituir valores
 *  Retirar caracteres especiais
*   Listar e executar!

PADRONIZAÇÃO DO CADASTRO

In [106]:
# Renomear as colunas para letras minúsculas e alterar o espaço por _
df_cadastro.columns = df_cadastro.columns.str.lower().str.replace(' ', '_')

# Subistituir valores texto para desconhecido caso não e exista e valor 0 para valores númericos
df_cadastro.fillna({
    'nome': 'DESCONHECIDO',
    'mae': 'DESCONHECIDO',
    'cpf': 0 ,
    'celular': 0,
    'email': 'DESCONHECIDO'
    }, inplace=True)

# Tabela de mapeamento para remoção de acentuação
accents_mapping = {
    'ã': 'a', 'â': 'a', 'á': 'a', 'à': 'a', 'ä': 'a',
    'ê': 'e', 'é': 'e', 'è': 'e', 'ë': 'e',
    'î': 'i', 'í': 'i', 'ì': 'i', 'ï': 'i',
    'ô': 'o', 'ó': 'o', 'ò': 'o', 'ö': 'o',
    'û': 'u', 'ú': 'u', 'ù': 'u', 'ü': 'u',
    'ç': 'c'
}

def remove_accents(text):
    for accent, replacement in accents_mapping.items():
        text = text.replace(accent, replacement)
    return text

# Aplicar a remoção de acentuação
text_columns = ['nome', 'mae']
for col in text_columns:
    df_cadastro[col] = df_cadastro[col].apply(remove_accents)
    df_cadastro[col] = df_cadastro[col].str.replace(r'[^\w\s]', '', regex=True)

# Alterar colunas com informações em texto para maiusculas
categorical_columns = ['nome', 'mae']
for col in categorical_columns:
    if col in df_cadastro.columns:
        df_cadastro[col] = df_cadastro[col].str.upper()

# Criar coluna com nome e sobrenome
df_cadastro['nome_array'] = df_cadastro.nome.str.split()
df_cadastro['primeiro_nome'] = df_cadastro.nome_array.str[0]
df_cadastro['sobrenome'] = df_cadastro.nome_array.str[-1]

# Converter o campo 'cpf' para string e preencher com zeros à esquerda para garantir 11 caracteres
if 'cpf' in df_cadastro.columns:  # Verifica se a coluna existe no DataFrame
    df_cadastro['cpf'] = df_cadastro['cpf'].astype(str).str.zfill(11)

# Converter a coluna 'data_nasc' para o tipo datetime
df_cadastro['data_nasc'] = pd.to_datetime(df_cadastro['data_nasc'], format='%d/%m/%Y')

# Criar coluna com ano, mês e dia separados
df_cadastro['nasc_ano'] = df_cadastro['data_nasc'].dt.year
df_cadastro['nasc_mes'] = df_cadastro['data_nasc'].dt.month.apply(lambda x: f'{x:02d}')
df_cadastro['nasc_dia'] = df_cadastro['data_nasc'].dt.day.apply(lambda x: f'{x:02d}')

# Exibir as primeiras 5 linhas
print("DataFrame após pré-processamento:")
print(df_cadastro.head().to_string(index=False))


DataFrame após pré-processamento:
                           nome  data_nasc         cpf                    mae   celular                                                                 email  telefone_fixo                           nome_array primeiro_nome sobrenome  nasc_ano nasc_mes nasc_dia
            CAUA ISAAC CALDEIRA 1945-03-02 35083528118 VITORIA KAMILLY SUELI  989048340               cauaisaaccaldeira__cauaisaaccaldeira@hardquality.com.br       28317237              [CAUA, ISAAC, CALDEIRA]          CAUA  CALDEIRA      1945       03       02
    LUCIA CAROLINA LUZIA ALMADA 1946-11-11 06480417194     STEFANY TERESINHA  997275887                                luciacarolinaluziaalmada_@santarte.com       27446850     [LUCIA, CAROLINA, LUZIA, ALMADA]         LUCIA    ALMADA      1946       11       11
    VICENTE MARCOS EDSON CASTRO 1959-01-15 72192158432 CECILIA LAURA LARISSA  994560743                          vicentemarcosedsoncastro-74@lavorosjc.com.br       28581372     [VI

PADRONIZAÇÃO DOS DADOS MÉDICOS

In [104]:
# Renomear as colunas para letras minúsculas e alterar o espaço por _
df_dados_medicos.columns = df_dados_medicos.columns.str.lower().str.replace(' ', '_')

# Subistituir valores texto para desconhecido caso não e exista e valor 0 para valores númericos
df_dados_medicos.fillna({
    'nome': 'DESCONHECIDO',
    'mae': 'DESCONHECIDO',
    'cpf': 0 ,
    'celular': 0,
    'email': 'DESCONHECIDO'
    }, inplace=True)

# Tabela de mapeamento para remoção de acentuação
accents_mapping = {
    'ã': 'a', 'â': 'a', 'á': 'a', 'à': 'a', 'ä': 'a',
    'ê': 'e', 'é': 'e', 'è': 'e', 'ë': 'e',
    'î': 'i', 'í': 'i', 'ì': 'i', 'ï': 'i',
    'ô': 'o', 'ó': 'o', 'ò': 'o', 'ö': 'o',
    'û': 'u', 'ú': 'u', 'ù': 'u', 'ü': 'u',
    'ç': 'c'
}

def remove_accents(text):
    for accent, replacement in accents_mapping.items():
        text = text.replace(accent, replacement)
    return text

# Aplicar a remoção de acentuação
text_columns = ['nome']
for col in text_columns:
    df_dados_medicos[col] = df_dados_medicos[col].apply(remove_accents)
    df_dados_medicos[col] = df_dados_medicos[col].str.replace(r'[^\w\s]', '', regex=True)

# Alterar colunas com informações em texto para maiusculas
categorical_columns = ['nome']
for col in categorical_columns:
    if col in df_dados_medicos.columns:
        df_dados_medicos[col] = df_dados_medicos[col].str.upper()

# Criar coluna com nome e sobrenome
df_dados_medicos['nome_array'] = df_dados_medicos.nome.str.split()
df_dados_medicos['primeiro_nome'] = df_dados_medicos.nome_array.str[0]
df_dados_medicos['sobrenome'] = df_dados_medicos.nome_array.str[-1]

# Converter o campo 'cpf' para string e preencher com zeros à esquerda para garantir 11 caracteres
if 'cpf' in df_dados_medicos.columns:  # Verifica se a coluna existe no DataFrame
    df_dados_medicos['cpf'] = df_dados_medicos['cpf'].astype(str).str.zfill(11)

# Converter a coluna 'data_nasc' para o tipo datetime
def parse_date(date_str):
    formats = ['%d/%m/%Y', '%d-%m-%Y']
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT  # Retorna NaT para formatos não reconhecidos

df_dados_medicos['data_nasc'] = df_dados_medicos['data_nasc'].apply(parse_date)

# Converter a coluna 'data_nasc' para o tipo datetime
df_dados_medicos['data_nasc'] = pd.to_datetime(df_dados_medicos['data_nasc'], format='%d/%m/%Y')

# Criar coluna com ano, mês e dia separados
df_dados_medicos['nasc_ano'] = df_dados_medicos['data_nasc'].dt.year
df_dados_medicos['nasc_mes'] = df_dados_medicos['data_nasc'].dt.month.apply(lambda x: f'{x:02d}')
df_dados_medicos['nasc_dia'] = df_dados_medicos['data_nasc'].dt.day.apply(lambda x: f'{x:02d}')

# Exibir as primeiras 5 linhas
print("DataFrame após pré-processamento:")
print(df_dados_medicos.head().to_string(index=False))

DataFrame após pré-processamento:
                       nome  data_nasc         cpf tipo_sanguineo  peso altura                       nome_array primeiro_nome sobrenome  nasc_ano nasc_mes nasc_dia
   ALICE LIZ ADRIANA DUARTE 1975-10-22 58548099313             O-    89   1,57    [ALICE, LIZ, ADRIANA, DUARTE]         ALICE    DUARTE      1975       10       22
            KAUE CESAR LIMA 1900-01-01 31515413217            AB-    62   1,99              [KAUE, CESAR, LIMA]          KAUE      LIMA      1900       01       01
OTAVIO FELIPE HENRY ALMEIDA 1951-01-10 58271711156             A-    85   1,66 [OTAVIO, FELIPE, HENRY, ALMEIDA]        OTAVIO   ALMEIDA      1951       01       10
   RAFAELA FRANCISCA MENDES 1974-11-07 31087464641             O+    46   1,74     [RAFAELA, FRANCISCA, MENDES]       RAFAELA    MENDES      1974       11       07
 BRYAN JOAQUIM ROBERTO ROSA 1957-09-07 61606247174            AB-    66   1,67  [BRYAN, JOAQUIM, ROBERTO, ROSA]         BRYAN      ROSA      1957 

Exercício 3: Indexação
*    Listar possíveis blocking Keys
*    Definir uma a ser utilizada
*    Criar os blocos
*    Listar os possíveis matches em cada bloco

LISTAR BLOCKING KEYS - Cadastro
* Consulta por CPF

In [85]:
# Criar os blocos com base na chave de bloqueio (cpf)
blocks = df_cadastro.groupby('cpf')

# Listar os possíveis matches em cada bloco
matches = []
for cpf, block in blocks:
    if len(block) > 1:
        block['block_cpf'] = cpf
        matches.append(block)

df_matches = pd.concat(matches)

print("Possíveis matches em cada bloco:")
print(df_matches.to_string(index=False))

Possíveis matches em cada bloco:
                     nome  data_nasc         cpf                        mae     celular                   email  telefone_fixo                    nome_array primeiro_nome sobrenome   block_cpf
MATHEUS ALBUQUERQUE SILVA 19/01/2015 58762423819 MARIANA SANTOS ALBUQUERQUE 11998970549 antonio.silva@gmail.com     1135494656 [MATHEUS, ALBUQUERQUE, SILVA]       MATHEUS     SILVA 58762423819
   ENZO ALBUQUERQUE SILVA 19/01/2015 58762423819 MARIANA SANTOS ALBUQUERQUE 11998970549 antonio.silva@gmail.com     1135494656    [ENZO, ALBUQUERQUE, SILVA]          ENZO     SILVA 58762423819
ANTONIO ALBUQUERQUE SILVA 24/07/1980 58762423819  LAURA ALBUQUERQUE CARDOSO 11998970549 antonio.silva@gmail.com     1135494656 [ANTONIO, ALBUQUERQUE, SILVA]       ANTONIO     SILVA 58762423819


DEFINIÇÃO DE REGRA:
* 6 PRIMEIROS DIGITOS DO CPF
* DATA DE NASCIMENTO SEM CARACTER
* E NOME

In [89]:
# Extrair os 6 primeiros dígitos do CPF
df_cadastro['cpf_prefix'] = df_cadastro['cpf'].str[:6]

# Remover caracteres da data de nascimento e criar nova coluna com a data no formato 'ddmmaaaa'
if 'data_nasc' in df_cadastro.columns:
    df_cadastro['data_nasc_clean'] = df_cadastro['data_nasc'].str.replace('-', '').str.replace('/', '')

# Criar a nova chave combinando CPF completo, Data de Nascimento limpa e Nome
df_cadastro['new_key'] = df_cadastro['cpf'] + df_cadastro['data_nasc_clean'] + df_cadastro['primeiro_nome']

# Criar os blocos com base na nova chave
blocks = df_cadastro.groupby('new_key')

# Listar os possíveis matches em cada bloco
matches = []
for key, block in blocks:
    if len(block) > 1:  # Só consideramos blocos com mais de um registro
        block['block_key'] = key  # Adiciona a chave de bloqueio ao DataFrame
        matches.append(block)

# Verificar o conteúdo de matches
print(f"Número de blocos com possíveis matches: {len(matches)}")
if matches:
    # Concatenar todos os blocos em um único DataFrame
    df_matches = pd.concat(matches, ignore_index=True)

    # Exibir a tabela com possíveis matches
    print("Possíveis matches em cada bloco:")
    print(df_matches.to_string(index=False))
else:
    print("Nenhum bloco com mais de um registro encontrado.")


Número de blocos com possíveis matches: 0
Nenhum bloco com mais de um registro encontrado.


LISTAR BLOCKING KEYS - Dados Médicos
* Consulta por CPF

In [92]:
# Criar os blocos com base na chave de bloqueio (cpf)
blocks = df_dados_medicos.groupby('cpf')

# Listar os possíveis matches em cada bloco
matches = []
for cpf, block in blocks:
    if len(block) > 1:
        block['block_cpf'] = cpf
        matches.append(block)

df_matches = pd.concat(matches)

print("Possíveis matches em cada bloco:")
print(df_matches.to_string(index=False))

Possíveis matches em cada bloco:
                          nome  data_nasc         cpf tipo_sanguineo  peso altura                          nome_array primeiro_nome  sobrenome cpf_prefix data_nasc_clean                       new_key   block_cpf
         BENICIO IAGO SILVEIRA 01/01/1900 00000000000             O+    97   1,72           [BENICIO, IAGO, SILVEIRA]       BENICIO   SILVEIRA     000000        01011900    0000000000001011900BENICIO 00000000000
         BETINA ELAINE DRUMOND 17/08/1995 00000000000            AB-    74   1,58           [BETINA, ELAINE, DRUMOND]        BETINA    DRUMOND     000000        17081995     0000000000017081995BETINA 00000000000
 SANDRA VITORIA LAVINIA BARROS 10/07/1958 00000000000            AB+    85   1,78  [SANDRA, VITORIA, LAVINIA, BARROS]        SANDRA     BARROS     000000        10071958     0000000000010071958SANDRA 00000000000
     PEDRO MARCOS TIAGO GALVAO 08/08/1987 00000000000             B-   101   1,91      [PEDRO, MARCOS, TIAGO, GALVAO]  

In [93]:
# Extrair os 6 primeiros dígitos do CPF
df_dados_medicos['cpf_prefix'] = df_dados_medicos['cpf'].str[:6]

# Remover caracteres da data de nascimento e criar nova coluna com a data no formato 'ddmmaaaa'
if 'data_nasc' in df_dados_medicos.columns:
    df_dados_medicos['data_nasc_clean'] = df_dados_medicos['data_nasc'].str.replace('-', '').str.replace('/', '')

# Criar a nova chave combinando CPF completo, Data de Nascimento limpa e Nome
df_dados_medicos['new_key'] = df_dados_medicos['cpf'] + df_dados_medicos['data_nasc_clean'] + df_dados_medicos['primeiro_nome']

# Criar os blocos com base na nova chave
blocks = df_dados_medicos.groupby('new_key')

# Listar os possíveis matches em cada bloco
matches = []
for key, block in blocks:
    if len(block) > 1:  # Só consideramos blocos com mais de um registro
        block['block_key'] = key  # Adiciona a chave de bloqueio ao DataFrame
        matches.append(block)

# Verificar o conteúdo de matches
print(f"Número de blocos com possíveis matches: {len(matches)}")
if matches:
    # Concatenar todos os blocos em um único DataFrame
    df_matches = pd.concat(matches, ignore_index=True)

    # Exibir a tabela com possíveis matches
    print("Possíveis matches em cada bloco:")
    print(df_matches.to_string(index=False))
else:
    print("Nenhum bloco com mais de um registro encontrado.")


Número de blocos com possíveis matches: 0
Nenhum bloco com mais de um registro encontrado.


Exercício 4: Comparação par a par
*    Desenvolver um algoritmo de comparação de cada match potencial do exercício anterior
*    Definir uma nota de similaridade para cada comparação
 *   Para cada parâmetro

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Função para calcular a similaridade entre dois registros
def calcular_similaridade(registro1, registro2):
    similaridade = {}

    # Comparar CPF (exemplo simples, você pode usar mais lógica para comparar CPFs)
    similaridade['cpf'] = 100 if registro1['cpf'] == registro2['cpf'] else 0

    # Comparar data de nascimento
    similaridade['data_nasc'] = 100 if registro1['data_nasc_clean'] == registro2['data_nasc_clean'] else 0

    # Comparar nome usando fuzzy matching
    similaridade['nome'] = fuzz.ratio(registro1['primeiro_nome'], registro2['primeiro_nome'])

    # Calcular a nota média de similaridade
    nota_media = sum(similaridade.values()) / len(similaridade)

    return similaridade, nota_media

# Comparar registros dentro de um bloco
def comparar_blocos(bloco):
    similaridade_results = []
    num_registros = len(bloco)

    # Comparar cada par de registros no bloco
    for i in range(num_registros):
        for j in range(i + 1, num_registros):
            registro1 = bloco.iloc[i]
            registro2 = bloco.iloc[j]

            similaridade, nota_media = calcular_similar


Exercício 5: Classificação
*    Classificar cada par, baseado num threshold
*    Definir um peso para cada parâmetro

In [ ]:
#Seu código aqui

Exercício 6: Avaliação
*     Buscar uma amostra das classificações (de 10 a 20 registros)
*     Classificar cada uma em TP, TN, FP, FN
*     Calcular acurácia, precisão e recall

In [ ]:
#Seu código aqui